# Instructions
 1. Change the variable definitions to the desired water quality station (site_no + station_no),  
 parameter (parametertype_shortname), and rainfall station timeseries (ts_id) data.
 2. Set time period (tmp) query to specify the period of interest.
 3. Set the parameters of interest. Not all parameters are available for all stations.
 4. Run the cell.  
 
To see what options are available, visit the [REST Services Directory](https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=getrequestinfo).  
&nbsp;    
  

### Commonly Used Parameters  

1. Common Timeseries parameters (ts_id):  
  * Rocky 24hr Rainfall: 25555010
  * Filucy 24hr Rainfall: 26223010
  * Purdy 24hr Rainfall: 26209010  
  * [View All](https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=getTimeseriesList&station_name=*)  
&nbsp;  
2. Common parameters (parametertype_shortename): 
  * pH, DO, DO_SaturationWaterTemperature, SpCond
  * FecalColiform_MF, Ecoli
  * [View All](https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=getWQMParameterList)  
&nbsp;  
3. Common sample (WQM) station IDs ([View All](https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=getWQMStationList)):
&nbsp;  

|     station_name   | station_no | station_latitude | station_longitude | site_no | site_name |
|---|---|---|---|---|---|
| KP_VaughnCreek_0.16 | KP_VAUG_0.16 | 47.345025 | -122.761138 | 15 | Kitsap |
| KP_RockyCreek_0.10 |	KP_ROCK_0.10 | 47.372565 | -122.781977 | 15 | Kitsap |
| KP_RockyCreek_1.20 |	KP_ROCK_1.20 | 47.38222 | -122.783362 | 15 | Kitsap |
| GH_PurdyCreek_0.14 |	GH_PURD_0.14 | 47.389211 | -122.625041 | 15 | Kitsap |
| KP_HugeCreek_0.25 | KP_HUGE_0.25 | 47.388999 | -122.698767 | 15 | Kitsap |
| KP_LittleMinterCreek_0.34 | KP_LMIN_0.34 | 47.381459 | -122.694931 | 15 | Kitsap |
| KP_MinterCreek_0.18 | KP_MINT_0.18 | 47.373251 | -122.702962 | 15 | Kitsap |  
| KP_SchoolHouseCreek_0.05 | KP_SCHO_0.05 |	47.226712 | -122.75104 | 15 | Kitsap |

   
&nbsp;  
### URL Queries
* Base Path:   
  * https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=
* Non-timeseries parameters as timeseries:  
  * getWqmSampleValuesAsTimeseries&period=*[Period]*&ts_path=*[Timeseries Path]*&format=csv&csvdiv=,&metadata=true
  * period can be form 'from + to', 'from', or 'PxYxMxD' (i.e. P36M for last 36 month). 
  * ts_path should be in the form: site_no/station_no/parametertype_shortname

* Timeseries for rainfall data:
  * 'getTimeseriesValues&'+*[Period]*+'&ts_id='+*[ts_id]*'&&format=csv&csvdiv=,&metadata=true'


In [2]:
import pyproj
import numpy as np
import datetime as dt
import pandas as pd
from bokeh.models import Circle, ColumnDataSource, Line, LinearAxis, Range1d, HoverTool
from bokeh.plotting import figure, output_file, show
from bokeh.core.properties import value
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.layouts import column, row, layout, gridplot
from bokeh.tile_providers import get_provider, Vendors

#-----> Data collection and processing: <-----#

# enter time period:
# for last 3 years, use 'period=P36M' (past 36 months) or 'period=P3Y' (past 3 years)
# or, for specific range (from x to y), use 'from=yyyy-mm-dd&to=yyyy-mm-dd'
# alternately, omit 'to=*' to to default to today's date ('from=yyyy-mm-dd')
tmp = 'period=P36M'

# enter sampling location (i.e. ts_path elements)
site_no = '15'
station_no = 'KP_VAUG_0.16'
parametertype_shortname = 'FecalColiform_MF'

# enter weather station ID for desired interval
ts_id = '25555010'

# set up url query for request - returns csv, but need to set csvdiv to ',' (defaults ';')
ts_path = site_no + '/' + station_no + '/' + parametertype_shortname
param_url = 'https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=getWqmSampleValuesAsTimeseries&'+tmp+'&ts_path='+ts_path+'&format=csv&csvdiv=,&metadata=true'

# set up url query for rainfall (timeseries)
rain_url = 'https://waterquality.piercecountywa.org/KiWIS/KiWIS?datasource=0&service=kisters&type=queryServices&request=getTimeseriesValues&'+tmp+'&ts_id='+ts_id+'&&format=csv&csvdiv=,&metadata=true'

# write datasets to pandas dataframes
# unless you specify '&metadata=false', need to skip the metadata rows
bdf = pd.read_csv(param_url, skiprows=9)
rdf = pd.read_csv(rain_url, skiprows=11).fillna(0)

# Format column header strings for clarity and consistency
tsp = 'Timestamp'
fecs = 'Bacteria'
rain = '24hr Rainfall (in)'

bdf.columns = [tsp, fecs]
rdf.columns = [tsp, rain]

# convert timestamp column values from string to datetime
rdf['Timestamp'] = pd.to_datetime(rdf[tsp])
bdf['Timestamp'] = pd.to_datetime(bdf[tsp])

# filter out nondetects to simplify graph (prettier, but unusable for statistical analysis) 
bdf1=bdf[bdf[fecs]!=1]

# filter out rainfall data that may predate sample data (if less than 36 months in dataset)
rdf1=rdf.loc[rdf[tsp] > bdf[tsp].iloc[0]]

# cast lat long coordinates for both stations from URL request to separate dataframe for map
rs_pts = pd.read_csv(rain_url, nrows=2, skiprows=1 )
bs_pts = pd.read_csv(param_url, nrows=2, skiprows=2)
pts = pd.DataFrame(
    {'site' : ['Weather_Station', 'WQ_Sample_Site'],
     'lat' : [rs_pts.iloc[0][1], bs_pts.iloc[0][1]],
     'long' : [rs_pts.iloc[1][1], bs_pts.iloc[1][1]]
    }
)

#----> Here is the vizualization process: <-----#


# set output HTML file name/location (CDN allows standalone document)
output_file("vaughn wqt report.html", mode="cdn")

# quick reference for column header vars
most_recent = str(bdf[tsp].iloc[-1])


# write fecal dataframe to bokey ColumnDataSource for custom tooltips
source = ColumnDataSource(bdf1)
tools = 'pan, wheel_zoom, save, box_zoom, reset'

# create a new plot (with a title) using figure
p = figure(x_axis_type='datetime', tools = tools, toolbar_location='above', 
           active_scroll='wheel_zoom', plot_width=1000, plot_height=450, y_axis_label='Fecal Coliforms (CFU/100 mL)',
           title=station_no+' Fecal Coliform Data as of '+most_recent[0:10]
          )

# Create 2nd y-axis first (so 1st axis prints on top)
p.extra_y_ranges['rain'] = Range1d(start=0, end=3)

p.add_layout(LinearAxis(y_range_name='rain', axis_label='24 Hour Rainall'), 'right')

# plot rain line graph
p.line(rdf1[tsp], rdf1[rain], legend='24hr Rainfall', y_range_name='rain', line_color='#6d6d6d', line_width=0.5)

# plot fecal sample scatter graph
fecal = p.scatter(x='Timestamp', y='Bacteria', size=11, fill_color='red', source=source, 
                  line_color='#F0F0F0', hover_line_color='black', legend='Fecal Coliform'
                 )

# plot regulatory reference lines
start = dt.timedelta(days=30)
end = dt.timedelta(days=1100)
s =(rdf1.iloc[0][0]-start)
doh = p.ray(x=s, y=43, source=source, legend='DOH Shellfish Limit (90th %-ile)', 
            color ='blue', line_dash='dashed', length=end, angle=0, line_width=0.5)
ecy = p.ray(x=s, y=100, source=source, legend='Ecology Recreation Limit', 
            color ='blue', line_dash='dashdot', length=end, angle=0, line_width=0.5)

# add/configure hover tooltip for fecal coliform 
#  (highlight style set in figure kwargs)
p_tips = [
    ('Fecal Count (CFU/100 mL): ','@Bacteria'),
    ('Sample Date: ', '@Timestamp{%F}')
    ]

p_columns = [
        TableColumn(field=tsp, title='Sample Date', formatter=DateFormatter()),
        TableColumn(field=fecs, title="Fecal Coliform (CFU/100 mL)"),
    ]

p.add_tools(HoverTool(tooltips=p_tips, formatters={'@Timestamp': 'datetime'}, renderers=[fecal]))

# add data table (interactive)
data_table = DataTable(source=source, columns=p_columns, width=400, height=1000,
                      selectable=True, sortable=True, row_height=22
                      )

# set up the map extent by passing coordinates (Accepts Mercator, not lat/long)
site_map = figure(x_range=(-13672358.13, -13656583.44), y_range=(5996570.8, 6004071.25), x_axis_type="mercator", y_axis_type="mercator",
                  tools = tools, toolbar_location='above', 
                  active_scroll='wheel_zoom', plot_width=1000, plot_height=450
                 )

# specify basemap (https://bokeh.pydata.org/en/latest/docs/reference/tile_providers.html#bokeh-tile-providers)
tile_provider = get_provider(Vendors.STAMEN_TERRAIN)
site_map.add_tile(tile_provider)

# convert lat long to mercator coords for map
outProj = pyproj.Proj(init='epsg:3857')
inProj = pyproj.Proj(init='epsg:4326')

r_x1 = pts.iloc[0][2]
r_y1 = pts.iloc[0][1]
b_x1 = pts.iloc[1][2]
b_y1 = pts.iloc[1][1]

merc_r = pyproj.transform(inProj, outProj, r_x1,r_y1)
merc_b = pyproj.transform(inProj, outProj, b_x1,b_y1)

pts_m = pd.DataFrame(
    {'site' : ['Weather_Station', 'WQ_Sample_Site'],
     'x' : [merc_r[0], merc_b[0]],
     'y' : [merc_r[1], merc_b[1]]
    }
)

# plot sample station location
pt_src = ColumnDataSource(pts_m[1:2])
site_map.circle(x='x', y='y', size=15, fill_color='blue', fill_alpha=0.8, source=pt_src)

# set up map tooltips
m_tips = [('Station Type: ', '@site')]
site_map.add_tools(HoverTool(tooltips=m_tips))

# set up and display the elements
show(row(column(p, site_map), data_table))